In [10]:
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [21]:
def clear():
    clear_output(wait=False)

In [25]:
a = [1,2,3]

In [27]:
x = pd.DataFrame(a)

In [33]:
int(input('Insert input'))
print(x)
int(input('Insert input'))

Insert input2
   0
0  1
1  2
2  3
Insert input2


2